In [11]:
from pyecharts.charts import Gauge
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
import pickle
import geocoder
from forecastiopy import *
import datetime

In [12]:
def cata (time,skin_type,spf,cloth_type,activities):
    
    g = geocoder.ip('me')
    a = g.latlng
    fio = ForecastIO.ForecastIO('7b455df1550a041b383e0cfbc5d2fb0c', latitude=a[0], longitude=a[1])
    current = FIOCurrently.FIOCurrently(fio)
    uv = current.uvIndex
    
    uvtime = [[uv,time]]
    
    if spf == 15:
        spf_par = 0.07
    elif spf == 30:
        spf_par = 0.03
    elif spf == 50:
        spf_par = 0.02
    elif spf == 0:
        spf_par = 1
        
        
    dose_model = pickle.load(open('dose_model.sav', 'rb'))
    logisticRegr_1 = pickle.load(open('logisticRegr_1.sav', 'rb'))    
    
    activities_dict = {'Picnic,Barbecue':1.05,'Grasslands':1.02,'Soil, clay':1.06,'Road trip':1.09,'City tour':1.22,
                    'Boat deck(wood or fibreglass)': 1.09,'Open water': 1.03,'Open ocean': 1.08,'Sea surf': 1.3,
                    'Beach sand, wet': 1.07,'Beach sand, dry': 1.18,'Snow':1.88}
    cloth_dict = {'Lycra/Elastanes':0.25,'Plastic':0.4,'Nylon':0.7,'Polyester':0.7,'Cotton':0.7,
                 'Polyester/Cotton':0.7}    
        
    poly = PolynomialFeatures(degree=2)
    uvtime_trans = poly.fit_transform(uvtime)
    does = dose_model.predict(uvtime_trans)*spf_par*activities_dict[activities]*cloth_dict[cloth_type]
    if does <= 0:
        does = 0
    
    dose_skin = np.array([does,skin_type])
    poly = PolynomialFeatures(degree=2)
    dose_skin = dose_skin.reshape(1,2)
    dose_skin_trans = poly.fit_transform(dose_skin)
    label = logisticRegr_1.predict(dose_skin_trans)
            
    return (label)

In [13]:
def spf_recom (time,skin_type,cloth_type,activities):
    spfli = [0,15,30,50]
    recommendedspf = 0
    for item in spfli:
        recommendedspf = item
        temp = float(cata(time,skin_type,item,cloth_type,activities))
        if temp < 2:
            break
    return(recommendedspf)

In [21]:
spf_recom(300,1,'Polyester/Cotton','Snow')

//anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0

In [16]:
cata(10,1,50,'Polyester/Cotton','Snow')

//anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


array([1])